# Version : 주차수요예측(제출)_법정주차수요

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
from sklearn.ensemble import RandomForestRegressor

# 데이터 불러오기

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')
gender = pd.read_csv('../data/age_gender_info.csv')

# 단지규모

In [3]:
# 파생변수 단지규모

# 소단지,중단지,대단지
danji_scale = pd.qcut(train['총세대수'], 3, labels=[1,2,3]) 
train.insert(14, '단지규모', danji_scale)

danji_scale = pd.qcut(test['총세대수'], 3, labels=[1,2,3]) 
test.insert(14, '단지규모', danji_scale)

# 전용면적당 법정주차대수 

## 법정주차대수 Column 추가

In [4]:
train.insert(loc=14, column='법정주차대수', value=0.0)
test.insert(loc=14, column='법정주차대수', value=0.0)

## 전용면적을 지역별 각 지역 기준으로 나누기

In [5]:
def PlaceParkingDivision(dataframe, place_colNames ,denominatorLessNum, denominatorExceedNum):
    df = dataframe.copy()
    
    for colName in place_colNames:
        idx = df[ (df['지역'] == colName) & (df['전용면적'] <= 85) ].index
        df.loc[idx, '법정주차대수'] = df[ (df['지역'] == colName) & (df['전용면적'] <= 85) ]['전용면적'] / denominatorLessNum
        idx = df[ (df['지역'] == colName) & (df['전용면적'] > 85) ].index
        df.loc[idx, '법정주차대수'] = df[ (df['지역'] == colName) & (df['전용면적'] > 85) ]['전용면적'] / denominatorExceedNum
        
    return df

In [6]:
# for문 columns 설정
Gwang_colNames = ['부산광역시', '대전광역시', '광주광역시', '울산광역시', '대구광역시', '세종특별자치시', '경기도']
Other_colNames = ['경상북도', '경상남도', '전라북도', '전라남도', '강원도', '충청남도', '충청북도', '제주특별자치도']

# Train
# 1. 서울특별시 
# - 전용면적 {85이하 - 75} | {85초과 - 65} 나누기
train = PlaceParkingDivision(train, ['서울특별시'], 75, 65)

# 2. 광역시 + 경기도
# - 전용면적 {85이하 - 85} | {85초과 - 70} 나누기
train = PlaceParkingDivision(train, Gwang_colNames, 85, 70)
    
# 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 전용면적 {85이하 - 95} | {85초과 - 75} 나누기
train = PlaceParkingDivision(train, Other_colNames, 95, 75)
    
# Test
# 2. 광역시 + 경기도
# - 전용면적 {85이하 - 85} | {85초과 - 70} 나누기
test = PlaceParkingDivision(test, Gwang_colNames, 85, 70)
    
# 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 전용면적 {85이하 - 95} | {85초과 - 75} 나누기
test = PlaceParkingDivision(test, Other_colNames, 95, 75)

## 법정주차대수 소수점 정리

In [7]:
def MyCeil(dataframe, place_colNames):
    df = dataframe.copy()
    
    for colName in place_colNames:
        idx = df[ df['지역'] == colName ].index
        df.loc[idx, '법정주차대수'] = np.ceil(df[ df['지역'] == colName ]['법정주차대수'])
        
    return df

In [8]:
# Train
# 1. 서울특별시 
# 1) 전용면적 30이하 : 0.5 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 30) ].index
train.loc[idx, '법정주차대수'] = 0.5

# 2) 전용면적 30초과 ~ 60이하 : 0.8 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 30) & (train['전용면적'] <= 60) ].index
train.loc[idx, '법정주차대수'] = 0.8

# 3) 전용면적 60초과 : 1.xxx -> 2 or 2.xxxx -> 3
# - 나머지가 조금이라도 나오면 몫의 값 + 1 => np.ceil 함수 사용
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ].index
train.loc[idx, '법정주차대수'] = np.ceil(train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ]['법정주차대수'])

# 2. 광역시 + 경기도
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
train = MyCeil(train, Gwang_colNames)

# 3. 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
train = MyCeil(train, Other_colNames)
    
# Test
# 2. 광역시 + 경기도
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
test = MyCeil(test, Gwang_colNames)

# 3. 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
test = MyCeil(test, Other_colNames)

# 총세대수 Log Scaling

In [9]:
train['총세대수'] = np.log(train['총세대수'])

# 컬럼명 바꾸기

In [10]:
train = train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

test = test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

# 결측치 처리

In [11]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
법정주차대수        0
단지규모          0
등록차량수         0
dtype: int64
-------- test.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
법정주차대수        0
단지규모          0
dtype: int64


In [12]:
# 1. 자격유형 D의 임대보증금/임대료를 자격유형 C의 임대보증금/임대료를 이용해서 값을 채우는 것이 적합한가?
# 2. 또 1번이 적절하다면, 차이가 큰 전용면적 52.74짜리의 임대보증금/임대료는 어떻게 채울 것인가?
# -> 해결책 : 결측치가 있는 지역과 같은 지역의 cost_per_meter=데이터에서 임대료(임대보증금)/전용면적 [단위:(원/m^2)]을 계산하여
#            결측치의 전용면적과 * cost_per_meter(원/m^2)로 값을 채운다.

# 임대건물구분이 '아파트'이면서 '임대보증금'&'임대료'가 결측치인 것들 처리해준다.

# Test
# m^2당 임대보증금/임대료를 구한다. 이때 단위는 (원/m^2) 이다
# 전용면적 최빈값(26.37)으로 나눠준다.
rental_cost_per_meter = 79980 / 26.37
deposit_cost_per_meter = 5787000 / 26.37 

# Test data에서 영구임대 지역은 대전광역시뿐이다.
# 즉, 아래 코드는 결측치를 적용하는 범위는 대전광역시이면서 공급유형이 영구임대인 곳이다.
# 결측치에 전용면적 * cost 변수(원/m^2)
idx = test[(test['공급유형']=='영구임대') & (test['임대보증금'].isnull())].index
test.loc[idx, '임대료'] = test[(test['공급유형']=='영구임대') & (test['임대보증금'].isnull())]['전용면적'] * rental_cost_per_meter
test.loc[idx, '임대보증금'] = test[(test['공급유형']=='영구임대') & (test['임대보증금'].isnull())]['전용면적'] * deposit_cost_per_meter

# 잘 들어갔는지 확인하자.
test.iloc[idx]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,법정주차대수,단지규모
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,5787000.0,79980.0,2.0,5.0,428.0,1.0,3
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,5787000.0,79980.0,2.0,5.0,428.0,1.0,3
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.0,D,11574000.0,159960.0,2.0,5.0,428.0,1.0,3


In [13]:
# Train 데이터에서는 임대건물구분이 '아파트'이지만 공급유형이 '공공분양'이기 때문에 이 다음 Cell에서 0으로 처리한다.
print('임대보증금 결측치 개수 : ', len(train[train['임대보증금'].isnull()]), '/ 임대료 결측치 개수 : ', len(train[train['임대료'].isnull()]))
print('아파트이면서 임대보증금이 결측치인 개수 : ', len(train[(train['임대건물구분']=='아파트') & (train['임대보증금'].isnull()==True)]))
print('상가이면서 임대보증금이 결측치인 개수 : ', len(train[(train['임대건물구분']=='상가') & (train['임대보증금'].isnull()==True)]))
train[(train['임대건물구분']=='아파트') & (train['임대보증금'].isnull()==True)]

임대보증금 결측치 개수 :  569 / 임대료 결측치 개수 :  569
아파트이면서 임대보증금이 결측치인 개수 :  7
상가이면서 임대보증금이 결측치인 개수 :  562


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,법정주차대수,단지규모,등록차량수
2331,C1350,7.244942,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2332,C1350,7.244942,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2333,C1350,7.244942,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2334,C1350,7.244942,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2335,C1350,7.244942,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2336,C1350,7.244942,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0
2337,C1350,7.244942,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,1.0,3,2315.0


In [14]:
# train 데이터
# 결측치는 (아파트 & 대전광역시 & 공공분양) + 상가인 데이터이다.
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)
train['지하철'] = train['지하철'].fillna(0)
train['버스'] = train['버스'].fillna(0)

# test 데이터
# 결측치는 모두 상가인 데이터이다.
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

In [15]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
법정주차대수      0
단지규모        0
등록차량수       0
dtype: int64
-------- test.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
법정주차대수      0
단지규모        0
dtype: int64


# '-' 데이터 처리

In [16]:
def HyphenDataProc(dataframe, demandName, qualificationName, getName, collection_range):
    df = dataframe.copy()
    cost_mean = []
    
    # 수집할 +/- 범위
    collection_range = collection_range
    
    # 자격유형 'H'이면서  임대료/임대보증금 '-'인 것들의 전용면적 가져오기
    area_list = df[(df[getName]=='-')&(df['자격유형']==qualificationName)]['전용면적'].to_list()

    # 조건문
    # '국민임대' & 'H' & '-' & 면적-0.5 이상 & 면적+0.5 이하 
    for area in area_list:
        cost_mean.append( np.mean(df[ (df['공급유형'] == demandName) & (df['자격유형'] == qualificationName) & (df[getName] != '-') 
          & (df['전용면적'] >= (area-collection_range)) & (df['전용면적'] <= (area+collection_range)) ][getName].values.astype(float)) )

    # 데이터 입력
    idx = df[(df[getName]=='-')&(df['자격유형']==qualificationName)].index
    df.loc[idx, getName] = cost_mean
    
    return df

## 장기전세 임대료 데이터 처리

In [17]:
idx = train[train['공급유형'] == '장기전세'].index
train.loc[idx, '임대료'] = 0.0

## 유형별 임대보증금/임대료 '-' 데이터 처리

In [18]:
# Train 공급유형 '국민임대' & 자격유형 'H' & 임대료/임대보증금 '-' 데이터 처리
train = HyphenDataProc(train, '국민임대', 'H', '임대료', 0.5)
train = HyphenDataProc(train, '국민임대', 'H', '임대보증금', 0.5)

# Train 공급유형 '행복주택' & 자격유형 'K' & 임대료/임대보증금 '-' 데이터 처리
train = HyphenDataProc(train, '행복주택', 'K', '임대료', 0.5)
train = HyphenDataProc(train, '행복주택', 'K', '임대보증금', 0.5)

# Test 공급유형 '행복주택' & 자격유형 'L' & 임대료/임대보증금 '-' 데이터 처리
test = HyphenDataProc(test, '행복주택', 'L', '임대료', 0.5)
test = HyphenDataProc(test, '행복주택', 'L', '임대보증금', 0.5)

# Test 공급유형 '영구임대' & 자격유형 'C' & 임대료/임대보증금 '-' 데이터 처리
test = HyphenDataProc(test, '영구임대', 'C', '임대료', 1.5)
test = HyphenDataProc(test, '영구임대', 'C', '임대보증금', 1.5)

# 임대료/임대보증금 Datatype 수치형으로 변경

In [19]:
# 임대료와 임대보증금이 object형이므로 수치형으로 변경해준다.
train['임대료'] = train['임대료'].astype('float64')
train['임대보증금'] = train['임대보증금'].astype('float64')

test['임대료'] = test['임대료'].astype('float64')
test['임대보증금'] = test['임대보증금'].astype('float64')

# 지하철 Y or N로 변경

In [20]:
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

# 임대건물구분 컬럼 처리

In [21]:
# 임대건물구분에서 7개의 아파트는 상가로 예상이됨 이를 처리 함.
# 임대건물구분이 상가인 경우는 전용면적별세대수도 1이므로 이를 처리 함.

idx = train[(train['임대건물구분']=='아파트') & (train['자격유형']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

# 지역 데이터를 숫자로 매핑

In [22]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

# 전용면적을 상/하한 적용

In [23]:
# 전용면적을 5의 배수로 변경 후 상/하한 적용
# 하한 값 15, 상한 값 100을 적용

train['전용면적'] = np.round(train['전용면적'], 0)
train['전용면적'] = train['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

test['전용면적'] = np.round(test['전용면적'], 0)
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 법정주차수요 만들기

In [24]:
value = train['총세대수'] * train['법정주차대수']
train.insert(loc=15, column='법정주차수요', value=value)

value = test['총세대수'] * test['법정주차대수']
test['법정주차수요'] = value

# 상관관계

In [25]:
pd.DataFrame(train.corr()['등록차량수'].sort_values(ascending=False))

,등록차량수
등록차량수,1.000000
단지내주차면수,0.859216
임대료,0.524180
임대보증금,0.436858
총세대수,0.382196
전용면적,0.354184
전용면적별세대수,0.239809
공가수,0.118121
버스,0.102293
지역,0.038256


In [26]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,법정주차대수,법정주차수요,단지규모,등록차량수
0,C2483,6.802395,아파트,0,국민임대,40.0,134,38.0,A,15667000.0,103680.0,0,3.0,1425.0,1.0,6.802395,2,1015.0
1,C2483,6.802395,아파트,0,국민임대,40.0,15,38.0,A,15667000.0,103680.0,0,3.0,1425.0,1.0,6.802395,2,1015.0
2,C2483,6.802395,아파트,0,국민임대,50.0,385,38.0,A,27304000.0,184330.0,0,3.0,1425.0,1.0,6.802395,2,1015.0
3,C2483,6.802395,아파트,0,국민임대,50.0,15,38.0,A,27304000.0,184330.0,0,3.0,1425.0,1.0,6.802395,2,1015.0
4,C2483,6.802395,아파트,0,국민임대,50.0,41,38.0,A,27304000.0,184330.0,0,3.0,1425.0,1.0,6.802395,2,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,5.476464,아파트,5,국민임대,45.0,19,7.0,A,11346000.0,116090.0,0,1.0,166.0,1.0,5.476464,1,146.0
2948,C2532,5.476464,아파트,5,국민임대,50.0,34,7.0,A,14005000.0,142310.0,0,1.0,166.0,1.0,5.476464,1,146.0
2949,C2532,5.476464,아파트,5,국민임대,50.0,34,7.0,A,14005000.0,142310.0,0,1.0,166.0,1.0,5.476464,1,146.0
2950,C2532,5.476464,아파트,5,국민임대,50.0,114,7.0,A,14005000.0,142310.0,0,1.0,166.0,1.0,5.476464,1,146.0


# 학습 데이터 생성

In [27]:
columns = ['총세대수', '지역', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수', '단지규모', '법정주차수요']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [28]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

new_train['임대보증금'] = 0
new_train['임대료'] = 0
new_test['임대보증금'] = 0
new_test['임대료'] = 0

In [29]:
# Train Dataframe Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0]) # index 재설정
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col] # Fixed Data
    
    deposit_cost = []
    rental_cost = []

    for col in area_columns:
        area = float(col.split('_')[-1]) # 면적의 숫자를 float형으로 저장
        # '면적_10' 등의 열을 만들고 - 같은 전용면적들 끼리 전용면적별세대수의 합을 Value로 Cell에 입력
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    # print(type(deposit_cost[0]))
    new_train.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_train.loc[i, '임대료'] = int(np.mean(rental_cost))

# Test DataFrame Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료, 등록차량수
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]

    deposit_cost = []
    rental_cost = []
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
          
    new_test.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_test.loc[i, '임대료'] = int(np.mean(rental_cost))

423it [00:11, 36.42it/s]
150it [00:04, 36.85it/s]


In [30]:
x_train = new_train.iloc[:, :-1]
y_train = new_train.iloc[:,-1]

x_test = new_test

In [31]:
forest = RandomForestRegressor(n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [32]:
pred = forest.predict(x_test)
submission['num'] = pred
display(submission)

,code,num
0,C1072,752.76
1,C1128,1392.80
2,C1456,541.13
3,C1840,543.52
4,C1332,1278.82
...,...,...
145,C2456,255.71
146,C1266,481.26
147,C2152,48.87
148,C1267,429.34


In [36]:
submission.to_csv('./법정주차수요.csv', index=False)